# Envibert Cased

In [28]:
from src.resources import hparams
from torch import nn
import torch

In [29]:
from importlib.machinery import SourceFileLoader
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import RobertaModel
from fairseq.models.roberta import XLMRModel
from src.model.bilstm import BiLSTM
from src.resources import hparams
from torch import nn
import os

In [30]:
bert_type = "xlmr"

In [31]:
if bert_type =="envibert_cased":
    print("use: envibert")
    tokenizer = SourceFileLoader("envibert.tokenizer", 
            os.path.join(hparams.toknizer_path,'envibert_tokenizer.py')).load_module().RobertaTokenizer(hparams.toknizer_path)
    bert = RobertaModel.from_pretrained('nguyenvulebinh/envibert',cache_dir=hparams.pretrained_envibert_cased)
elif bert_type =="envibert_uncased":
    print("use: envibert_uncased")
    tokenizer = SourceFileLoader("envibert.tokenizer", 
            os.path.join(hparams.pretrained_envibert_uncased,'envibert_tokenizer.py')).load_module().RobertaTokenizer(hparams.pretrained_envibert_uncased)
    bert = XLMRModel.from_pretrained(hparams.pretrained_envibert_uncased, checkpoint_file='model.pt')
elif bert_type =="xlmr":
    print("use: xlm roberta")
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    bert = XLMRobertaModel.from_pretrained("xlm-roberta-base")
    

use: xlm roberta


In [32]:
class bert_bilstm(nn.Module):
    def __init__(self, nb_label, cuda, bert, drop_rate, hidden_dim_lstm, hidden_dim_bert):
        super().__init__()
        self.bert = bert
        self.bilstm = BiLSTM(
            cuda=cuda, 
            embedding_dim=hidden_dim_bert, 
            hidden_dim=hidden_dim_lstm
            )
        self.dropout = nn.Dropout(drop_rate)
        self.linear = nn.Linear(hidden_dim_lstm, nb_label).to(cuda)

    def forward(self, input_ids, input_masks):
        output = self.bert(input_ids=input_ids, 
                        attention_mask = input_masks)
        sequence_output, _ = output[0], output[1]
        
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.bilstm(sequence_output)
        output = self.linear(sequence_output)

        return output

In [33]:
model = bert_bilstm(
        cuda="cuda:1",
        nb_label=hparams.nb_labels, 
        bert=bert,
        drop_rate=hparams.drop_rate,
        hidden_dim_bert=768,
        hidden_dim_lstm=hparams.hidden_dim_lstm).to("cuda:1")

In [34]:
tmp = torch.load("/home/tuyendv/projects/text-restoration-model/checkpoint/xlmr/checkpoint_xlmr_1.pt")

In [35]:
res = {}
for key, value in tmp.items():
    res[key.replace("module.","")] = value
model.load_state_dict(res)

<All keys matched successfully>

In [27]:
torch.save(model.state_dict(),"checkpoint_xlmr_1_chatbot.pt")

In [12]:
model = torch.load("/home/tuyendv/projects/restore-text/checkpoint_xlmr_3.pt", map_location="cpu")

In [13]:
input_ids = torch.tensor([[1,2,3,4]])
label_ids = torch.tensor([[1,1,1,1]])

In [14]:
model(input_ids ,label_ids)

TypeError: 'collections.OrderedDict' object is not callable

# Envibert uncased

In [ ]:
from src.resources import hparams
from torch import nn
import torch

In [ ]:
from importlib.machinery import SourceFileLoader
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import RobertaModel
from fairseq.models.roberta import XLMRModel
from src.model.bilstm import BiLSTM
from src.resources import hparams
from torch import nn
import os